In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [10]:
print("pine cone key", os.getenv("PINECONE_API_KEY"))

pine cone key 5c02fdb0-26f0-4159-bd42-a19e2d840e11


In [9]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [6]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Olivia Nguyen',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Professor Nguyen is an excellent lecturer who makes complex topics accessible. Her assignments are challenging but fair, and she is always available to provide extra help during office hours.'},
 {'professor': 'Professor Michael Goldstein',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': 'While Professor Goldstein is knowledgeable about the subject matter, his lectures can be dry and he does not provide much guidance on how to approach the challenging problem sets. The exams are also very difficult.'},
 {'professor': 'Dr. Sarah Patel',
  'subject': 'Art History',
  'stars': 5,
  'review': 'Dr. Patel is passionate about art and her enthusiasm is infectious. Her lectures are engaging and she encourages students to think critically about the works we study. The reading load is manageable, and the assignments are interesting and thought-provoking.'},
 {'professor': 'P

In [7]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-3-small',
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })


In [ ]:
processed_data[0]

In [10]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

{'upserted_count': 127}

In [11]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 127}},
 'total_vector_count': 127}